In [9]:
import glob
from PIL import Image
import numpy as np
from nntool.api import NNGraph

In [10]:
quant_model_path = "model/cifar10_model_uint8.tflite"

G = NNGraph.load_graph(quant_model_path, load_quantization=True)

G.quantize(
    statistics=None,
    graph_options={
        "ne16": False,
        "hwc": False,
    }
)
G.adjust_order()
G.add_dimensions()
G.fusions('scaled_match_group')

indicating change of FULLY_CONNECTED_0_11 input from c, out_cin_c, out_c to chw, out_cin_chw, out_c order - rerun adjust command
indicating change of FULLY_CONNECTED_0_11 output from c to chw order - rerun adjust command
indicating change of FULLY_CONNECTED_0_12 input from c, out_cin_c, out_c to chw, out_cin_chw, out_c order - rerun adjust command
indicating change of FULLY_CONNECTED_0_12 output from c to chw order - rerun adjust command


In [11]:
dataset_files = glob.glob("samples/*")
for file in dataset_files:
	print(file)
	img = Image.open(file)
	# Transpose HWC to CHW
	input_tensor = np.array(img).transpose((2, 0, 1))
	# Even in quantized mode nntool requires float data exactly as you trained the model -> [-1.0: 1.0]
	input_tensor = input_tensor.astype(np.float32) / 128 - 1

	outputs = G.execute([input_tensor], quantize=True, dequantize=True)
	pred_class = np.argmax(outputs[-1][0])
	confidence = outputs[-1][0][pred_class]
	print(f"Predicted Class: {pred_class} with confidence: {confidence}")


samples/cifar_test_13_7.ppm
Predicted Class: 7 with confidence: 0.994598388671875
samples/cifar_test_16_5.ppm
Predicted Class: 5 with confidence: 0.89288330078125
samples/cifar_test_1_8.ppm
Predicted Class: 8 with confidence: 0.9962158203125
samples/cifar_test_20_7.ppm
Predicted Class: 7 with confidence: 0.86663818359375
samples/cifar_test_3_0.ppm
Predicted Class: 8 with confidence: 0.672271728515625
samples/cifar_test_24_5.ppm
Predicted Class: 4 with confidence: 0.896820068359375
samples/cifar_test_12_5.ppm
Predicted Class: 5 with confidence: 0.545867919921875
samples/cifar_test_8_3.ppm
Predicted Class: 3 with confidence: 0.935821533203125
samples/cifar_test_0_3.ppm
Predicted Class: 3 with confidence: 0.914398193359375
samples/cifar_test_17_7.ppm


KeyboardInterrupt: 